# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [4]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplleaflet
import pandas as pd
import calendar
import datetime


binsize = 400
hashid = "fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89"

def get_data():
    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))
    return df[df['hash'] == hashid]

def leaflet_plot_stations():
    station_locations_by_hash = get_data()
    
    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()
    
    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

# leaflet_plot_stations()

In [5]:
# Select only intersted columns and creates Day and Month columns
def prepare_dataset(df):
    df = df[["ID", "Date", "Element", "Data_Value"]].copy()
    df['Day'] = df["Date"].dt.day.values
    df['Month'] = df["Date"].dt.month.values
    return df

In [6]:
# Delete Leap Days
def remove_leap(df):
    return df[~((df["Day"] == 29) & (df["Month"] == 2))]

In [7]:
df = get_data()

temp_df = pd.read_csv("data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")
temp_df.reset_index(inplace=True)
temp_df = df.merge(temp_df, how='left', on="ID")

temp_df["Data_Value"] = temp_df["Data_Value"] / 10
# convert date columnt to datetime type
temp_df["Date"] = pd.to_datetime(temp_df["Date"])

# line graph of the record high and record low temperatures by day of the year over the period 2005-2014. 
# The area between the record high and record low temperatures for each day should be shaded.

# get only data between 2005 and 2014
temp_df_05_14 = temp_df[(temp_df["Date"]>'2004') & (temp_df["Date"]<'2015')].copy()

# get only data for 2015
temp_df_15 = temp_df[(temp_df["Date"]>'2014') & (temp_df["Date"]<'2016')].copy()


# Prepare Datasets
temp_df_05_14 = prepare_dataset(temp_df_05_14)
temp_df_15 = prepare_dataset(temp_df_15)

# Delete Leap Days
temp_df_05_14 = remove_leap(temp_df_05_14)
temp_df_15 = remove_leap(temp_df_15)

# Calulate Max and Min temps datasets
dfMin_05_14, dfMax_05_14 = get_max_min(temp_df_05_14)
dfMin_15, dfMax_15 = get_max_min(temp_df_15)

#
dfMin_15 = dfMin_15[dfMin_15["Data_Value"] < dfMin_05_14["Data_Value"]]
dfMax_15 = dfMax_15[dfMax_15["Data_Value"] > dfMax_05_14["Data_Value"]]

plt.figure(figsize=(10, 5))
plt.title("The interval of minimum and maximum temperatures for 2005-2014 \n and temperatures of 2015, dropping out of the interval",
          family='sans-serif',
          size=11
         )

# Line PLot
plt.plot_date(dfMax_05_14.index.values, dfMax_05_14["Data_Value"], '-r', label='Record High 2005-2014', alpha=0.8, lw=0.7)
plt.plot_date(dfMin_05_14.index.values, dfMin_05_14["Data_Value"], '-b', label='Record Low 2005-2014', alpha=0.8, lw=0.7)

plt.fill_between(dfMax_05_14.index.values, dfMax_05_14["Data_Value"].values, dfMin_05_14["Data_Value"].values, facecolor='whitesmoke', alpha=0.8)

# Scatter Plot
plt.plot_date(dfMax_15.index.values, dfMax_15["Data_Value"], 'or', label='2015 above Max')
plt.plot_date(dfMin_15.index.values, dfMin_15["Data_Value"], 'ob', label='2015 below Min')


# align plot start and finish with axis start and finish
plt.xlim(datetime.date(1900, 1, 1), datetime.date(1900, 12, 31))

# Remove top and right box
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_color("dimgrey")
ax.spines['left'].set_color("dimgrey")
ax.spines['bottom'].set_alpha(0.5)
ax.spines['left'].set_alpha(0.5)

# Set months names on x axis

# Set the locator
locator = mdates.MonthLocator()  # every month
# Specify the format - %b gives us Jan, Feb...
fmt = mdates.DateFormatter('%b')

X = ax.xaxis
X.set_major_locator(locator)
# Specify formatter
X.set_major_formatter(fmt)

# Create a Legend
leg = plt.legend()
leg.get_frame().set_linewidth(0.0)

plt.ylabel("Temperature(°C)", color="dimgrey", alpha=0.8)

plt.show()

NameError: name 'get_max_min' is not defined